<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Classification/CNN_Classification_automatique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#-----Imports et installations-----
#Pour se connecter au drive et acceder a ses fichiers
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#Pour l'apprentissage profond
import os
from fastai.vision.all import *
from fastai.vision import *
import pandas as pd

Mounted at /content/drive


In [2]:
Espece_visee = "Solcan"
Chemin_donnees = "/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis"

In [ ]:
Noms_classes = []
for Noms_dossiers in os.listdir(Chemin_donnees):
    if ("." in Noms_dossiers) or (Noms_dossiers=="models"):
        continue
    else:
        Noms_classes.append(Noms_dossiers)
Noms_classes

In [6]:
Chemins_images = []
Etiquettes = []

for i in range(len(Noms_classes)):
    Classe_nom = Noms_classes[i]
    if "." in Classe_nom or Classe_nom=="models":
       continue
    else:        
        Noms_images = os.listdir(f"{Chemin_donnees}/{Classe_nom}")
        for Image_nom in Noms_images:
            Image_chemin = f"{Chemin_donnees}/{Classe_nom}/{Image_nom}"
            Chemins_images.append(Image_chemin)
            Etiquettes.append(i)

In [8]:
df = pd.DataFrame(
    {
        "Nom": Chemins_images,
        "Etiquette": Etiquettes
    }
)

,Nom,Etiquette
0,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Fleuri/468542.jpeg,0
1,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Fleuri/177001.jpeg,0
2,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Fleuri/177002.jpeg,0
3,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Fleuri/176611.jpeg,0
4,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Fleuri/266540181.jpeg,0
...,...,...
995,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Trop proche/250161201.jpeg,3
996,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Trop proche/250161063.jpeg,3
997,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Trop proche/250161168.jpeg,3
998,/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis/Trop proche/250161042.jpeg,3


In [ ]:
df.to_csv(
    path_or_buf=f"{Chemin_donnees}/etiquettes.csv",
    index=False
)

Telechargement des donnees

In [ ]:
dls = ImageDataLoaders.from_folder(
      path=Chemin_donnees,
      item_tfms=Resize(224),
      bs=16,
      batch_tfms=[Normalize.from_stats(*imagenet_stats), RandTransform()],
      valid_pct=0.2
)

Creation du modele

In [ ]:
Modele = vision_learner(
    dls=dls,
    arch=models.resnet18,
    metrics=[accuracy, error_rate]
)

In [ ]:
Nb_epochs = 1
Performance = []

for epoch in range(Nb_epochs):
    Modele.fit_one_cycle(1)
    Modele.save(f"CNN_{Espece_visee}_epoch{epoch}.pth")
    Perfo = Modele.recorder.final_record
    Performance.append(Perfo)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.198279,1.065572,0.770000,0.230000,03:56
